# L2HMC using eager execution in tensorflow

### Imports

In [ ]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.special import i0, i1

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from l2hmc_eager import dynamics_eager as _l2hmc
from l2hmc_eager import gauge_dynamics_eager as l2hmc
from l2hmc_eager.neural_nets import *
from utils.distributions import GMM, gen_ring
from utils.jacobian import _map, jacobian

%autoreload 2

In [ ]:
from lattice.ising_lattice import IsingLattice
from lattice.gauge_lattice import GaugeLattice

In [ ]:
tf.enable_eager_execution()
tfe = tf.contrib.eager

In [ ]:
tfe = tf.contrib.eager

In [ ]:
def train_one_iter(dynamics, beta, x, optimizer, 
                   loss_fn=l2hmc.compute_loss, global_step=None):
    loss, grads, out, accept_prob = l2hmc.loss_and_grads(
        dynamics, x, loss_fn=loss_fn
    )
    optimizer.apply_gradients(
        zip(grads, dynamics.trainable_variables), global_step=global_step
    )
    return loss, out, accept_prob

In [ ]:
def exact_plaquette_average(beta):
    return i1(beta) / i0(beta)

### Construct GaugeLattice with $U(1)$ gauge group

$$ U_{\mu\nu} = \frac{\beta}{3}\sum_{\nu \neq \mu} \mathrm{Re}\left\{\mathrm{Tr}\left[U_{\mu}(x)U_{\nu}(x+\hat\mu)U_{\mu}^{\dagger}(x+\hat\nu)U_{\nu}^{\dagger}(x)\right]\right\}$$

#### Heatbath Algorithm

In [ ]:
time_size, space_size, dim, beta, num_samples = (16, 16, 2, 4., 5)
u1_lattice = GaugeLattice(time_size, space_size, dim, beta,
                          link_type='U1', num_samples=num_samples)
u1_samples = [sample.flatten() for sample in u1_lattice.samples]
u1_samples_tensor = tf.constant(np.stack(u1_samples), dtype=tf.float32)

In [ ]:
eq_steps = 5000
acceptances = []
action_arr = [u1_lattice._total_action()]
avg_plaq_arr = [u1_lattice._average_plaquette()]
for i in range(eq_steps):
    action = u1_lattice._total_action()
    avg_plaq = u1_lattice._average_plaquette()
    change = avg_plaq - avg_plaq_arr[-1]
    avg_plaq_arr.append(avg_plaq)
    action_arr.append(action)
    print(f"Step: {i:<5g}\t action: {action:<8.4g}\t "
          f"avg plaq: {avg_plaq:<8.4g}\t change: {change:<8.4g}")
    accept = 0
    for site in u1_lattice.iter_sites():
        for d in range(u1_lattice.dim):
            accept += u1_lattice._update_link(site, d)
    acceptances.append(accept)
# 12.2s for 500 equilibration steps

In [ ]:
np.mean(avg_plaq_arr[:100])

In [ ]:
np.mean(avg_plaq_arr)

In [ ]:
exact_plaquette_average(beta)

In [ ]:
avg_plaq_arr = [0]
p = k - j
for k in range(j, 40000):
    avg_plaq = u1_lattice._average_plaquette()
    change = avg_plaq - avg_plaq_arr[p-1]
    avg_plaq_arr.append(avg_plaq)
    print(f"Step: {k:<5g}: avg plaq: {avg_plaq:>12.4g} change: {change:12.4g}")
    for site in u1_lattice.iter_sites():
        for d in range(u1_lattice.dim):
            _ = u1_lattice._update_link(site, d)

In [ ]:
num_acceptances = 0
measure_steps = 10000
avg_plq = np.zeros(measure_steps)
for step in range(measure_steps):
    for site in u1_lattice.iter_sites():
        for d in range(u1_lattice.dim):
            num_acceptances += u1_lattice._update_link(site, d)
    avg_plq[step] = u1_lattice._average_plaquette()

In [ ]:
u1_lattice._total_action()

In [ ]:
u1_lattice._average_plaquette()

#### Run L2HMC for $U(1)$ gauge model

In [ ]:
time_size, space_size, dim, beta, num_samples = (8, 8, 2, 2., 10)
u1_lattice = GaugeLattice(time_size, space_size, dim, beta,
                          link_type='U1', num_samples=num_samples, rand=False)
#u1_samples = u1_lattice.get_links_samples(batch_size, link_type='U1')
#u1_samples_flat = [i.flatten() for i in u1_samples]
u1_samples = [sample.flatten() for sample in u1_lattice.samples]
u1_samples_tensor = tf.constant(np.stack(u1_samples), dtype=tf.float32)

In [ ]:
# Construct dynamics object
u1_energy_fn = u1_lattice.get_energy_function(u1_samples_tensor)
u1_dynamics = l2hmc.GaugeDynamics(u1_lattice, 
                                  minus_loglikelihood_fn=u1_energy_fn, 
                                  n_steps=10, eps=0.05)

In [ ]:
################  tests  #############################
#u1_lattice._total_action()
#u1_lattice._average_plaquette()
#u1_lattice.total_action(u1_samples)
#u1_lattice.total_action()
#u1_lattice.average_plaquette(u1_samples)
#u1_lattice.average_plaquette()
#---------------------------------------------
#_momentum = tf.random_normal(tf.shape(u1_samples))
#_potential = np.array(u1_dynamics.potential(u1_samples_tensor))
#_kinetic = u1_dynamics.kinetic(_momentum)
#_grad_potential = u1_dynamics.grad_potential(u1_samples_tensor)
#print(_potential); print('\n')
#print(_kinetic); print('\n')
#print(_grad_potential)
#print(_kinetic.numpy()); print('\n')
#print(_hamiltonian.numpy()); print('\n')
#print(_grad_potential[0][:10])
#---------------------------------------------
#site = u1_lattice.get_random_site()
#u = np.random.randint(u1_lattice.dim)
#v = np.random.randint(u1_lattice.dim)
#plaq = u1_lattice.plaquette_operator(site, u, v)
#---------------------------------------------
#u1_lattice.total_action(u1_samples_tensor)
#u1_lattice.average_plaquette(u1_samples_tensor)
#transition_out = u1_dynamics.apply_transition(u1_samples_tensor)
#x_post, p_post, accept_prob, x_out = transition_out
#loss, x_out, x_accept_prob = l2hmc.compute_loss(u1_dynamics, u1_samples_tensor)
#x_accept_prob
###############################################

In [ ]:
# Create new log_dir with new run number
log_dirs = os.listdir('../../U1_logs/')
run_nums = [int(i.split('_')[-1]) for i in log_dirs if i.startswith('run')]
run_num = max(run_nums) + 1
log_dir = f'../../U1_logs/run_{run_num}'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
print(log_dir)

In [ ]:
global_step = tf.train.get_or_create_global_step()
_ = global_step.assign(1)
train_iters = 500
record_loss_every = 10
save_steps = 50 

learning_rate = tf.train.exponential_decay(1e-3, global_step, 1000, 
                                           0.96, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate)
checkpointer = tf.train.Checkpoint(
    optimizer=optimizer, dynamics=u1_dynamics, global_step=global_step
)
summary_writer = tf.contrib.summary.create_file_writer(log_dir)
loss_fn = l2hmc.compute_loss
samples = u1_samples_tensor

In [ ]:
total_actions = []
average_plaquettes = []
t0 = time.time()
for i in range(1, 500):
    t1 = time.time()
    loss, samples, accept_prob = train_one_iter(
        u1_dynamics,
        u1_lattice.beta,
        samples,
        optimizer,
        loss_fn=loss_fn,
        global_step=global_step
    )
    _total_actions = u1_lattice.total_action(samples)
    _avg_plaquettes = u1_lattice.average_plaquette(samples)
    total_actions.extend(_total_actions)
    average_plaquettes.extend(_avg_plaquettes)

    print("Iteration {}, loss {:.4f}, x_accept {:.4f},"
          " eps {:.4f}, avg_S {:.4f}, avg_plaquette: {:.4f}".format(
              i, loss.numpy(), accept_prob.numpy().mean(),
              u1_dynamics.eps.numpy(), np.mean(_total_actions), 
              np.mean(_avg_plaquettes))
         )
    print(f'\n _avg_plaquettes: {[i.numpy() for i in _avg_plaquettes]}\n')
    print(f'time per training step: {time.time() - t1}\n')

    if i % record_loss_every == 0:
        with summary_writer.as_default():
            with tf.contrib.summary.always_record_summaries():
                tf.contrib.summary.scalar("Training loss", loss,
                                          step=global_step)

    if i % save_steps == 0:
        saved_path = checkpointer.save(file_prefix=os.path.join(log_dir,
                                                                "ckpt"))
        print(f"Saved checkpoint to: {saved_path}")

In [ ]:
exact_plaquette_average(u1_lattice.beta)

In [ ]:
u1_lattice.total_action(samples)

In [ ]:
samples[0]

In [ ]:
plaqs = []
S = []
S_approx = []
for site in _latt.iter_sites():
    for mu in range(_latt.dim):
        for nu in range(_latt.dim):
            if nu > mu:
                plaq =  _latt.plaquette_operator(_latt.links, site, mu, nu)
                _S = 1 - np.cos(plaq)
                S_approx.append(0.5 * plaq ** 2)
                plaqs.append(plaq)
                S.append(_S)

In [ ]:
saved_path = checkpointer.save(file_prefix=os.path.join(train_dir, 
                                                        "ckpt"))
print(f"Saved checkpoint to: {saved_path}")

In [ ]:
import sys
help(sys.stdout.flush)

### Construct GaugeLattice with SU(3) gauge group

In [ ]:
time_size = 2
space_size = 4
dim = 4
beta = 1.
link_type = 'SU3' 
batch_size = 3
gauge_lattice = GaugeLattice(time_size, space_size, dim, beta, link_type)
# create `num_samples` random samples of GaugeLattice.links
links_samples = gauge_lattice.get_links_samples(batch_size, link_type=link_type)

In [ ]:
gauge_energy_fn = gauge_lattice.get_energy_function()
gauge_dynamics = l2hmc.GaugeDynamics(gauge_lattice, 
                                     minus_loglikelihood_fn=gauge_energy_fn, 
                                     batch_size=3, n_steps=5, eps=0.1)

In [ ]:
gauge_lattice.links.shape

In [ ]:
potential_arr = gauge_dynamics.potential(links_samples, batch_size)

[i.numpy() for i in potential_arr]

In [ ]:
_momentum = tf.random_normal(tf.shape(links_samples))

In [ ]:
gauge_dynamics.kinetic(_momentum).numpy()

In [ ]:
_x = links_samples
#_momentum = tf.random_normal(tf.shape(_x))
_hamiltonian = gauge_dynamics.hamiltonian(_x, _momentum)
_hamiltonian

### Construct IsingLattice

In [ ]:
ising_batch_size = 10
ising_lattice = IsingLattice(3, 4)
ising_samples = [ising_lattice._randomize() for _ in range(ising_batch_size)]

In [ ]:
ising_energy_fn = ising_lattice.get_energy_function()
ising_dynamics = l2hmc.LatticeDynamics(ising_lattice, 
                                       minus_loglikelihood_fn=ising_energy_fn,
                                       batch_size=ising_batch_size, 
                                       n_steps=10, eps=0.1)
#dynamics = l2hmc.LDynamics(latt.sites.shape, minus_loglikelihood_fn=energy_fn, n_steps=10, eps=0.1)

In [ ]:
ising_dynamics.potential(samples, batch_size)

In [ ]:
_iposition = ising_samples
_imomentum = tf.random_normal(tf.shape(_iposition))
_ihamiltonian = dynamics.hamiltonian(_iposition, _imomentum)
_ihamiltonian

In [ ]:
_isample = _iposition[0].reshape(ising_lattice.num_sites)
#dynamics.grad_potential(np.array(_position).reshape(-1, lattice.num_sites))

In [ ]:
grad_pot = dynamics.grad_potential(ising_samples)

In [ ]:
grad_pot

In [ ]:
ising_jacobian = jacobian(dynamics.potential, ising_samples)

In [ ]:
%debug

In [ ]:
grad_fn = tfe.gradients_function(lattice._calc_energy, params=[0])

In [ ]:
_jacobian = jacobian(dynamics.potential, _position)

In [ ]:
%debug

In [ ]:
lattice.calc_energy(_position, batch_size)

In [ ]:
#dynamics.position_fn(momentum, latt.sites.flatten()[:], dynamics)
#dynamics._forward_lf(latt.sites.flatten()[:], momentum, 0)
dynamics._forward_lf(np.array(_position).reshape(-1, lattice.num_sites),
                     np.array(_momentum).reshape(-1, lattice.num_sites), 1)

### GMM Model

In [ ]:
sigmas, distribution = gen_ring(1., var=0.02, nb_mixtures=4)

gmm_potential = distribution.get_energy_function()
gmm_dynamics = _l2hmc.Dynamics(x_dim=2, minus_loglikelihood_fn=gmm_potential,
                               n_steps=25, eps=0.1)

In [ ]:
samples = distribution.get_samples(200)

In [ ]:
_position = samples
_momentum = tf.random_normal(tf.shape(_position))
_hamiltonian = gmm_dynamics.hamiltonian(_position, _momentum)

In [ ]:
grad_pot = gmm_dynamics.grad_potential(_position, _momentum)

In [ ]:
grad_pot.shape